# Hilbert-Huang Transform (HHT) 解析チュートリアル

このチュートリアルでは **gwexpy** の `TimeSeries.hht()` を用いた **推奨ワークフロー** を紹介します。
HHT は非線形・非定常な信号に強く、STFT やウェーブレットでは捉えにくい瞬時周波数の変化を可視化できます。

HHT は次の2ステップで構成されます:
1. **Empirical Mode Decomposition (EMD/EEMD)**: 信号を IMF と残差に分解
2. **Hilbert Spectral Analysis**: 各 IMF の瞬時振幅・瞬時周波数を算出

**注意**: 本チュートリアルには `PyEMD` (EMD-signal) が必要です。
```bash
pip install EMD-signal
```


In [1]:
import matplotlib.pyplot as plt
import numpy as np

from gwexpy.timeseries import TimeSeries

# Check PyEMD availability
try:
    import PyEMD

    print(f"{PyEMD.__name__} is installed and ready.")
except ImportError:
    raise ImportError(
        "This tutorial requires 'PyEMD' (EMD-signal). Please run: pip install EMD-signal"
    )


/home/runner/miniconda3/envs/docs/lib/python3.11/site-packages/gwpy/time/_ligotimegps.py:42: UserWarning: Wswiglal-redir-stdio:

SWIGLAL standard output/error redirection is enabled in IPython.
This may lead to performance penalties. To disable locally, use:

with lal.no_swig_redirect_standard_output_error():
    ...

To disable globally, use:

lal.swig_redirect_standard_output_error(False)

Note however that this will likely lead to error messages from
LAL functions being either misdirected or lost when called from
Jupyter notebooks.

To suppress this warning, use:

import warnings
warnings.filterwarnings("ignore", "Wswiglal-redir-stdio")
import lal

  from lal import LIGOTimeGPS


AttributeError: module 'gwpy.io.registry' has no attribute 'register_reader'

## 1. シミュレーションデータの作成

以下の非定常信号を合成します:
- 低周波サイン (5 Hz)
- チャープ (80 → 120 Hz)
- 線形トレンド


In [2]:
# 時間軸 (1秒間、1000サンプル)
t = np.linspace(0, 1, 1000)
dt = t[1] - t[0]

# 信号成分
s1 = 0.5 * np.sin(2 * np.pi * 5 * t)  # 低周波
s2 = 1.0 * np.sin(2 * np.pi * 80 * t * (1 + 0.5 * t))  # チャープ (80Hz -> 120Hz)
trend = 2.0 * t  # 線形トレンド

# TimeSeries の作成
data = TimeSeries(s1 + s2 + trend, dt=dt, unit="V", name="Simulation Data")

# プロット
plot = data.plot(title="Original Simulation Signal")
plot.show()


NameError: name 'TimeSeries' is not defined

## 2. 推奨ワークフロー: `TimeSeries.hht()`

`TimeSeries.hht()` は EMD/EEMD とヒルベルト解析を一括で実行します。
ここでは次を指定します:
- `emd_kwargs`: EEMD 設定（チュートリアルでは試行回数を少なめに設定）
- `hilbert_kwargs`: パディングと平滑化で IF を安定化

**Tip**: EEMD は確率的なので、再現性のため `random_state` を指定します。


In [3]:
emd_kwargs = {
    "eemd_trials": 20,
    "random_state": 42,
    "sift_max_iter": 200,
    "stopping_criterion": 0.2,
}
hilbert_kwargs = {
    "pad": 100,
    "if_smooth": 11,
}

result = data.hht(
    emd_method="eemd",
    emd_kwargs=emd_kwargs,
    hilbert_kwargs=hilbert_kwargs,
    output="dict",
)

imfs = result["imfs"]
plot = imfs.plot(figsize=(10, 10), sharex=True, title="IMFs (EEMD)")
plot.show()

plt.figure(figsize=(10, 4))
for key in list(imfs.keys())[:3]:
    if_ts = result["if"][key]
    plt.plot(if_ts.times.value, if_ts.value, label=key)

plt.xlabel("Time [s]")
plt.ylabel("Instantaneous Frequency [Hz]")
plt.title("Instantaneous Frequency (First 3 IMFs)")
plt.legend()
plt.show()


NameError: name 'data' is not defined

### 解釈（IMFs と IF）
第1 IMF が高周波チャープを主に表し、瞬時周波数が時間とともに増加します。


## 3. ヒルベルトスペクトル (HHTSpectrogram)

`output='spectrogram'` を指定すると時間-周波数マップを得られます。`HHTSpectrogram.plot()` は既定で対数スケールです。


In [4]:
spec = data.hht(
    output="spectrogram",
    emd_method="eemd",
    emd_kwargs=emd_kwargs,
    hilbert_kwargs=hilbert_kwargs,
    fmin=0,
    fmax=200,
    n_bins=120,
    weight="ia2",
)

plot = spec.plot(figsize=(10, 5))
plot.show()


NameError: name 'data' is not defined

## 4. 低レベルAPIについて（任意）

詳細制御が必要な場合は以下を直接使用できます:
- `emd()`（分解）
- `hilbert_analysis()`（IA/IF）
- `instantaneous_frequency()` / `envelope()`（個別 IMF）

通常の解析では `hht()` が推奨です。


### ヒント
- 端点効果が出やすいので `hilbert_kwargs={'pad': N}` を検討し、端は無視してください。
- より滑らかな IMF が必要なら `eemd_trials` を増やします（計算時間増）。
- 再現性重視なら `emd_method='emd'` を使います。


### まとめ
`TimeSeries.hht()` を使うと IMF の診断とヒルベルトスペクトルの両方を簡潔に取得できます。
